In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import time
import seaborn as sns
import gc
import os 
import random
from datetime import datetime as dt
from pytz import timezone
from sklearn.model_selection import ParameterGrid
import matplotlib.pyplot as plt
%matplotlib inline

# Mount Google drive

In [ ]:
root_path = "/users/hailey/data/05_conference_workshop_grant/2023_KNU_bootcamp/results"



# Load input data

In [2]:
data_file = '/users/hailey/data/05_conference_workshop_grant/2023_KNU_bootcamp/FC_input.npz'
fc_data = np.load(data_file)

X = fc_data['x'] 
X = np.arctanh(X)
X = stats.zscore(X, axis=1)
y = fc_data['y'].astype(np.float_).copy().reshape(-1, 1)

print(X.shape,y.shape,sbjs.shape)

(3200, 19900) (3200, 1) (3200,)


In [6]:
sbjs = fc_data['sbjs']
all_sbjs = np.unique(sbjs)
np.random.shuffle(all_sbjs)

train_sbjs = all_sbjs[0:80]
val_sbjs = all_sbjs[80:90]
test_sbjs = all_sbjs[90:100]

train_sbjs_loc = np.in1d(sbjs,train_sbjs)
valid_sbjs_loc = np.in1d(sbjs,val_sbjs)
test_sbjs_loc = np.in1d(sbjs,test_sbjs)

[ 37  15  54  93  72  67  66   5  33  35  68  83  10  19  94  20  63  47
  31  42  51 100  55  81  70   4  75  23  78  28  56  29   6  30  14  91
  44  86  25  45  97  22  95  73  24  89   1  90  59   2  77  18  43  61
  13  57  99  98  41  34   8  46  76  88  65  53   7  80  52  40  21  36
  16  26  85  48   9  50  39  71] [32 79 92 96 69 82 38  3 27 49] [62 17 84 74 60 11 58 87 64 12]
(1, 3200)
(1, 3200)
(1, 3200)


# Set hyperparameters

In [5]:
seed = 42

learning_rate = 1e-5
min_lr = 1e-10
lr_patience = 10
momentum = 0.90

l2_param = 1e-5
l1_param_cand = [1e-3,5e-4,1e-4,5e-5,1e-5]
param_cand = {l1_param": l1_param_cand}
param_grid = list(ParameterGrid(param_cand))
param_grid
              
h1 = 512
h2 = 512

dropout_h1 = 0.4
dropout_h2 = 0.4

batch_size = 128
epochs = 200
print_epoch = 10

input_dim = X.shape[1]
output_dim = 1

[{'l1_param': 0.001, 'learning_rate': 5e-05},
 {'l1_param': 0.001, 'learning_rate': 3e-05},
 {'l1_param': 0.001, 'learning_rate': 1e-05},
 {'l1_param': 0.001, 'learning_rate': 5e-06},
 {'l1_param': 0.0005, 'learning_rate': 5e-05},
 {'l1_param': 0.0005, 'learning_rate': 3e-05},
 {'l1_param': 0.0005, 'learning_rate': 1e-05},
 {'l1_param': 0.0005, 'learning_rate': 5e-06},
 {'l1_param': 0.0001, 'learning_rate': 5e-05},
 {'l1_param': 0.0001, 'learning_rate': 3e-05},
 {'l1_param': 0.0001, 'learning_rate': 1e-05},
 {'l1_param': 0.0001, 'learning_rate': 5e-06},
 {'l1_param': 5e-05, 'learning_rate': 5e-05},
 {'l1_param': 5e-05, 'learning_rate': 3e-05},
 {'l1_param': 5e-05, 'learning_rate': 1e-05},
 {'l1_param': 5e-05, 'learning_rate': 5e-06},
 {'l1_param': 1e-05, 'learning_rate': 5e-05},
 {'l1_param': 1e-05, 'learning_rate': 3e-05},
 {'l1_param': 1e-05, 'learning_rate': 1e-05},
 {'l1_param': 1e-05, 'learning_rate': 5e-06}]

# Setup Pytorch

In [ ]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ReduceLROnPlateau 
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

### Set CPU/GPU usage

In [7]:
num_workers = 4

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu") 

### Random seed

In [8]:
def seed_everything(seed=seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(seed)

# Define functions

### Define functions for data load

In [9]:
class GetDataset(Dataset): 
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx): 
        X = torch.from_numpy(self.X[idx]).type(torch.FloatTensor)
        y = torch.from_numpy(self.y[idx]).type(torch.FloatTensor)

        return X, y

### Define functions for training/validaiton/test

In [11]:
class DNN(nn.Module):
    def __init__(self, h1, h2, dropout_h1, dropout_h2):
        # 2 hidden layers
        super(DNN, self).__init__()
        self.ext_1 = nn.Linear(input_dim, h1)
        self.ext_bn_1 = nn.BatchNorm1d(h1)
        
        self.prd_1 = nn.Linear(h1, h2)
        self.prd_bn_1 = nn.BatchNorm1d(h2)
        self.prd_2 = nn.Linear(h2, output_dim)
        
        self.dropout_h1 = nn.Dropout(p=dropout_h1)
        self.dropout_h2 = nn.Dropout(p=dropout_h2)
        
        self.act_func = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.weights_init()
    
    def forward(self, x):
        x = self.ext_1(x)
        x = self.ext_bn_1(x)
        x = self.act_func(x)
        x = self.dropout_h1(x)
        
        x = self.prd_1(x)
        x = self.prd_bn_1(x)
        x = self.act_func(x)
        x = self.dropout_h2(x)
        x = self.prd_2(x)         
        out = self.sigmoid(x)
        
        return out
    
    def weights_init(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode="fan_in", nonlinearity="relu")
                nn.init.normal_(m.bias, std=0.01)

In [10]:
def train(model, epoch, train_loader, optimizer, criterion, l1_param, l2_param):
    model.train()
    train_loss = 0
    train_acc = 0
    cost = 0
    correct = 0
    total = 0
    
    for batch_idx, (input, target) in enumerate(train_loader):
        optimizer.zero_grad() 
        input, target = input.to(device), target.to(device)
        pred, target = model(input), target.view(-1, 1)
        
        running_loss = criterion(pred, target)
        all_linear1_params = torch.cat([x.view(-1) for x in model.ext_1.parameters()])
        all_linear2_params = torch.cat([x.view(-1) for x in model.prd_1.parameters()])
        l1_regularization = l1_param * (torch.norm(all_linear1_params, 1) + torch.norm(all_linear2_params, 1))
        l2_regularization = l2_param * (torch.norm(all_linear1_params, 2) + torch.norm(all_linear2_params, 2))

        cost = running_loss + l1_regularization + l2_regularization
        cost.backward()
        optimizer.step()
        train_loss += running_loss.item()
        total += pred.size(0)
                
        crite = torch.FloatTensor([0.5])
        crite = crite.to(device)
        prediction = pred >= crite  # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == target # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도 계산
        correct += accuracy
        
    train_loss = train_loss / total
    train_acc = correct / (batch_idx+1)

    return train_loss, train_acc

In [12]:
def valid(model, epoch, valid_loader, criterion):
    model.eval()
    valid_loss = 0
    valid_acc = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for input, target in valid_loader:
            input, target = input.to(device), target.to(device)
            pred, target = model(input), target.view(-1, 1)
            running_loss = criterion(pred, target)
            valid_loss += running_loss.detach()
            total += pred.size(0)
            
            crite = torch.FloatTensor([0.5])
            crite = crite.to(device)
            prediction = pred >= crite # 예측값이 0.5를 넘으면 True로 간주
            correct_prediction = prediction.float() == target # 실제값과 일치하는 경우만 True로 간주
            accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도 계산
            correct += accuracy

    valid_acc = correct
    
    return valid_loss, valid_acc 

In [13]:
def test(model, epoch, test_loader, criterion):
    model.eval()
    test_loss = 0
    test_acc = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for input, target in test_loader:
            input, target = input.to(device), target.to(device)
            pred, target = model(input), target.view(-1, 1)
            running_loss = criterion(pred, target)
            test_loss += running_loss.item()
            total += pred.size(0)
            
            crite = torch.FloatTensor([0.5])
            crite = crite.to(device)
            prediction = pred >= crite  # 예측값이 0.5를 넘으면 True로 간주
            correct_prediction = prediction.float() == target # 실제값과 일치하는 경우만 True로 간주
            accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
            correct += accuracy
            
    test_acc = correct

    return test_loss, test_acc 

### Define function for model training

In [15]:
def run_train(params, i_model=0, rst_path=None, val_flg=0, print_epoch=10):

    start_time = time.time()

    X_train, y_train = X[train_sbjs_loc,:], y[train_sbjs_loc]
    X_valid, y_valid = X[valid_sbjs_loc,:], y[valid_sbjs_loc]
    X_test, y_test = X[test_sbjs_loc,:], y[test_sbjs_loc]
            
    train_dataset = GetDataset(X_train, y_train)
    valid_dataset = GetDataset(X_valid, y_valid)
    test_dataset = GetDataset(X_test, y_test)
    
    train_loader = DataLoader(
        train_dataset, batch_size=batch_size, pin_memory=True,
        shuffle=True, num_workers=num_workers, drop_last=True)
    valid_loader = DataLoader(
        valid_dataset, batch_size=len(y_valid), pin_memory=True,
        shuffle=True, num_workers=num_workers, drop_last=True)
    test_loader = DataLoader(
        test_dataset, batch_size=len(y_test), pin_memory=True,
        shuffle=True, num_workers=num_workers, drop_last=True)
        
    # Assign model 
    model = DNN(h1, h2, dropout_h1, dropout_h2).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, nesterov=True)
    scheduler = ReduceLROnPlateau(
        optimizer, mode="min", patience=lr_patience, min_lr=min_lr
    )
    criterion = nn.functional.binary_cross_entropy
              
    # list to save learning parameters
    train_loss_list = []
    valid_loss_list = []
    train_acc_list = []
    valid_acc_list = []
    test_acc_list = []

    for epoch in range(1, epochs + 1):
        train_loss, train_acc = train(
            model, epoch, train_loader, optimizer, criterion, params['l1_param'], l2_param
        )
        valid_loss, valid_acc = valid(model, epoch, valid_loader, criterion)
        test_loss, test_acc = test(model, epoch, test_loader, criterion)

        scheduler.step(valid_loss)
        lr = optimizer.param_groups[0]['lr']
        
        train_loss_list.append(train_loss)
        train_acc_list.append(train_acc)
        valid_loss_list.append(valid_loss)
        valid_acc_list.append(valid_acc)
        
        if epoch % print_epoch == 0:
            print("Epoch [{:d}/{:d}]".format(epoch, epochs), end=" ")
            print("Lr: {:.1e}, Train loss: {:.4f}, Valid loss: {:.4f}, Train acc: {:.2f}, Valid acc: {:.2f}"
                  .format(lr, train_loss, valid_loss, train_acc, valid_acc))

            plot_learning_curves(
                rst_path, epochs, train_loss_list, valid_loss_list,  
                train_acc_list, valid_acc_list, i_model, val_flg=1
            )
    
    torch.save(model.state_dict(), 
               rst_path + "/model_" + str(i_model+1) + ".pt")
    
    torch.cuda.empty_cache()
    gc.collect()
    
    acc_list = [train_acc, valid_acc, test_acc]
            
    acc_df = pd.DataFrame(np.array(acc_list), columns=["train_acc", "valid_acc", "test_acc"])
    acc_df.to_csv("{}/acc.txt".format(rst_path))

    tot_time = time.time() - start_time
    print("\nExecution Time for model: {:.2f} mins".format(tot_time / 60))
    
    return train_acc, valid_acc, test_acc

### Define functions for saving results

In [14]:
def plot_learning_curves(
    rst_path, epochs, train_loss, valid_loss, train_acc, valid_acc, i_model=0, val_flg=0):
    
    sns.set(style="darkgrid", font_scale=2)
    fig, ax = plt.subplots(1, 2, figsize=(18, 10))
    ax = ax.flat
    lw = 4
    last_epoch = epochs
    
    ax[0].plot(train_loss[:last_epoch], label='train loss', lw=lw, color="r")
    if val_flg:
        ax[0].plot(valid_loss[:last_epoch], label='valid loss', lw=lw, color="g")
    else:
        ax[0].plot(test_loss[:last_epoch], label='test loss', lw=lw, color="g")
    ax[0].set_yscale('log')
    ax[0].legend()
    ax[0].set_title("Loss Plot", pad=10)

    ax[1].plot(train_acc[:last_epoch], label='train acc', lw=lw, color="r")
    if val_flg:
        ax[1].plot(valid_acc[:last_epoch], label='valid acc', lw=lw, color="g")
    else:
        ax[1].plot(test_acc[:last_epoch], label='test acc', lw=lw, color="g")
    ax[1].legend()
    ax[1].set_title("Accuracy Plot", pad=10)
    
    fig.tight_layout()
    fig.savefig("{}/Learning_curves_model_{}.png".format(rst_path,i_model+1))
    
    plt.close(fig)

# Create directory & save model info

In [1]:
nowtime = dt.now(timezone("Asia/Seoul"))

year = str(nowtime.year)
month = '0{}'.format(nowtime.month) if nowtime.month < 10 else str(nowtime.month)
day = '0{}'.format(nowtime.day) if nowtime.day < 10 else str(nowtime.day)
hour = '0{}'.format(nowtime.hour) if nowtime.hour < 10 else str(nowtime.hour)
minute = '0{}'.format(nowtime.minute) if nowtime.minute < 10 else str(nowtime.minute)
sec = '0{}'.format(nowtime.second) if nowtime.second < 10 else str(nowtime.second)

save_path = os.path.join(root_path, "result_{}{}{}_{}{}{}".format(year,month,day,hour,minute,sec))
os.makedirs(save_path, exist_ok=True)
rst_path = "{}/train_rst".format(save_path)
os.makedirs(rst_path, exist_ok=True)


f = open(save_path+"/info.txt",'w')
f.write('data_file : '+data_file+'\n')
f.write('seed : '+str(seed)+'\n')
f.write('learning_rate : '+str(learning_rate)+'\n')
f.write('min_lr : '+str(min_lr)+'\n')
f.write('lr_patience : '+str(lr_patience)+'\n')
f.write('momentum : '+str(momentum)+'\n')
f.write('l2_param : '+str(l2_param)+'\n')
f.write('dropout_h1 : '+str(dropout_h1)+'\n')
f.write('dropout_h2 : '+str(dropout_h2)+'\n')
f.write('batch_size : '+str(batch_size)+'\n')
f.write('input_dim : '+str(input_dim)+'\n')
f.write('output_dim : '+str(output_dim)+'\n')
f.write('epochs : '+str(epochs)+'\n')
f.close()
np.savez(save_path+'/sbj_shuffle',train_sbjs=train_sbjs,val_sbjs=val_sbjs,test_sbjs=test_sbjs)

print(save_path)

NameError: name 'dt' is not defined

# Start training

In [ ]:

all_acc_list = []
for i_model,params in enumerate(param_grid):

    print("\n<{}> {}".format(i_model+1,params))
    train_acc_model, valid_acc_model, test_acc_model = run_train(
        params, i_model, rst_path, val_flg = 0, print_epoch=print_epoch
    )
    all_acc_list.append([params, train_acc_model, valid_acc_model, test_acc_model])


all_valid_acc = [item[2] for item in all_acc_list]
selected_model = np.argmax(all_valid_acc)

print("\n<{}> Selected {}, train acc: {:.4f}, valid acc: {:.4f}, test acc: {:.4f}"
      .format(selected_model+1, param_grid[selected_model], all_acc_list[selected_model][1], all_acc_list[selected_model][2], all_acc_list[selected_model][3]))


=================================== Outer Fold [1/1] ===================================

<1> {'l1_param': 0.001, 'learning_rate': 5e-05}
(2560, 19900) (320, 19900) (320, 19900)
Epoch [10/1000] Lr: 5.0e-05, Train loss: 0.0036, Valid loss: 0.3552, Train acc: 0.79, Valid acc: 0.86
Epoch [20/1000] Lr: 5.0e-05, Train loss: 0.0027, Valid loss: 0.2831, Train acc: 0.85, Valid acc: 0.88
Epoch [30/1000] Lr: 5.0e-05, Train loss: 0.0023, Valid loss: 0.2482, Train acc: 0.88, Valid acc: 0.89
Epoch [40/1000] Lr: 5.0e-05, Train loss: 0.0020, Valid loss: 0.2252, Train acc: 0.90, Valid acc: 0.90
Epoch [50/1000] Lr: 5.0e-05, Train loss: 0.0018, Valid loss: 0.2099, Train acc: 0.92, Valid acc: 0.91
Epoch [60/1000] Lr: 5.0e-05, Train loss: 0.0015, Valid loss: 0.1982, Train acc: 0.93, Valid acc: 0.91
Epoch [70/1000] Lr: 5.0e-05, Train loss: 0.0014, Valid loss: 0.1890, Train acc: 0.94, Valid acc: 0.92
Epoch [80/1000] Lr: 5.0e-05, Train loss: 0.0013, Valid loss: 0.1809, Train acc: 0.94, Valid acc: 0.92
Epoch

Epoch [790/1000] Lr: 5.0e-09, Train loss: 0.0004, Valid loss: 0.1372, Train acc: 0.99, Valid acc: 0.95
Epoch [800/1000] Lr: 5.0e-09, Train loss: 0.0003, Valid loss: 0.1374, Train acc: 0.99, Valid acc: 0.94
Epoch [810/1000] Lr: 5.0e-09, Train loss: 0.0003, Valid loss: 0.1370, Train acc: 1.00, Valid acc: 0.95
Epoch [820/1000] Lr: 5.0e-09, Train loss: 0.0003, Valid loss: 0.1376, Train acc: 1.00, Valid acc: 0.94
Epoch [830/1000] Lr: 5.0e-09, Train loss: 0.0004, Valid loss: 0.1370, Train acc: 0.99, Valid acc: 0.95
Epoch [840/1000] Lr: 5.0e-09, Train loss: 0.0003, Valid loss: 0.1375, Train acc: 1.00, Valid acc: 0.94
Epoch [850/1000] Lr: 5.0e-09, Train loss: 0.0003, Valid loss: 0.1374, Train acc: 0.99, Valid acc: 0.94
Epoch [860/1000] Lr: 5.0e-09, Train loss: 0.0003, Valid loss: 0.1380, Train acc: 1.00, Valid acc: 0.94
Epoch [870/1000] Lr: 5.0e-09, Train loss: 0.0003, Valid loss: 0.1372, Train acc: 0.99, Valid acc: 0.94
Epoch [880/1000] Lr: 5.0e-09, Train loss: 0.0003, Valid loss: 0.1373, Tra

Epoch [580/1000] Lr: 3.0e-09, Train loss: 0.0006, Valid loss: 0.1543, Train acc: 0.98, Valid acc: 0.93
Epoch [590/1000] Lr: 3.0e-09, Train loss: 0.0006, Valid loss: 0.1544, Train acc: 0.98, Valid acc: 0.93
Epoch [600/1000] Lr: 3.0e-09, Train loss: 0.0005, Valid loss: 0.1543, Train acc: 0.99, Valid acc: 0.93
Epoch [610/1000] Lr: 3.0e-09, Train loss: 0.0005, Valid loss: 0.1546, Train acc: 0.98, Valid acc: 0.93
Epoch [620/1000] Lr: 3.0e-09, Train loss: 0.0006, Valid loss: 0.1545, Train acc: 0.98, Valid acc: 0.93
Epoch [630/1000] Lr: 3.0e-09, Train loss: 0.0005, Valid loss: 0.1543, Train acc: 0.99, Valid acc: 0.93
Epoch [640/1000] Lr: 3.0e-09, Train loss: 0.0005, Valid loss: 0.1543, Train acc: 0.99, Valid acc: 0.93
Epoch [650/1000] Lr: 3.0e-09, Train loss: 0.0006, Valid loss: 0.1542, Train acc: 0.98, Valid acc: 0.93
Epoch [660/1000] Lr: 3.0e-09, Train loss: 0.0006, Valid loss: 0.1545, Train acc: 0.99, Valid acc: 0.93
Epoch [670/1000] Lr: 3.0e-09, Train loss: 0.0006, Valid loss: 0.1542, Tra

Epoch [370/1000] Lr: 1.0e-05, Train loss: 0.0014, Valid loss: 0.2073, Train acc: 0.94, Valid acc: 0.91
Epoch [380/1000] Lr: 1.0e-05, Train loss: 0.0014, Valid loss: 0.2054, Train acc: 0.94, Valid acc: 0.91
Epoch [390/1000] Lr: 1.0e-05, Train loss: 0.0014, Valid loss: 0.2035, Train acc: 0.94, Valid acc: 0.92
Epoch [400/1000] Lr: 1.0e-05, Train loss: 0.0014, Valid loss: 0.2016, Train acc: 0.94, Valid acc: 0.92
Epoch [410/1000] Lr: 1.0e-05, Train loss: 0.0014, Valid loss: 0.2013, Train acc: 0.94, Valid acc: 0.91
Epoch [420/1000] Lr: 1.0e-05, Train loss: 0.0014, Valid loss: 0.1990, Train acc: 0.94, Valid acc: 0.91
Epoch [430/1000] Lr: 1.0e-05, Train loss: 0.0013, Valid loss: 0.1973, Train acc: 0.94, Valid acc: 0.92
Epoch [440/1000] Lr: 1.0e-05, Train loss: 0.0013, Valid loss: 0.1954, Train acc: 0.95, Valid acc: 0.91
Epoch [450/1000] Lr: 1.0e-05, Train loss: 0.0013, Valid loss: 0.1939, Train acc: 0.94, Valid acc: 0.91
Epoch [460/1000] Lr: 1.0e-05, Train loss: 0.0013, Valid loss: 0.1923, Tra

Epoch [160/1000] Lr: 5.0e-06, Train loss: 0.0030, Valid loss: 0.3145, Train acc: 0.83, Valid acc: 0.88
Epoch [170/1000] Lr: 5.0e-06, Train loss: 0.0030, Valid loss: 0.3084, Train acc: 0.83, Valid acc: 0.88
Epoch [180/1000] Lr: 5.0e-06, Train loss: 0.0029, Valid loss: 0.3021, Train acc: 0.83, Valid acc: 0.87
Epoch [190/1000] Lr: 5.0e-06, Train loss: 0.0027, Valid loss: 0.2963, Train acc: 0.85, Valid acc: 0.88
Epoch [200/1000] Lr: 5.0e-06, Train loss: 0.0026, Valid loss: 0.2912, Train acc: 0.86, Valid acc: 0.88
Epoch [210/1000] Lr: 5.0e-06, Train loss: 0.0027, Valid loss: 0.2856, Train acc: 0.85, Valid acc: 0.88
Epoch [220/1000] Lr: 5.0e-06, Train loss: 0.0026, Valid loss: 0.2815, Train acc: 0.86, Valid acc: 0.89
Epoch [230/1000] Lr: 5.0e-06, Train loss: 0.0026, Valid loss: 0.2772, Train acc: 0.86, Valid acc: 0.89
Epoch [240/1000] Lr: 5.0e-06, Train loss: 0.0025, Valid loss: 0.2732, Train acc: 0.86, Valid acc: 0.89
Epoch [250/1000] Lr: 5.0e-06, Train loss: 0.0026, Valid loss: 0.2689, Tra

...

Epoch [920/1000] Lr: 5.0e-06, Train loss: 0.0011, Valid loss: 0.1891, Train acc: 0.96, Valid acc: 0.93
Epoch [930/1000] Lr: 5.0e-06, Train loss: 0.0011, Valid loss: 0.1885, Train acc: 0.95, Valid acc: 0.93
Epoch [940/1000] Lr: 5.0e-06, Train loss: 0.0011, Valid loss: 0.1878, Train acc: 0.95, Valid acc: 0.93
Epoch [950/1000] Lr: 5.0e-06, Train loss: 0.0010, Valid loss: 0.1868, Train acc: 0.96, Valid acc: 0.93
Epoch [960/1000] Lr: 5.0e-06, Train loss: 0.0011, Valid loss: 0.1865, Train acc: 0.96, Valid acc: 0.93
Epoch [970/1000] Lr: 5.0e-06, Train loss: 0.0010, Valid loss: 0.1861, Train acc: 0.96, Valid acc: 0.93
Epoch [980/1000] Lr: 5.0e-06, Train loss: 0.0010, Valid loss: 0.1854, Train acc: 0.97, Valid acc: 0.93
Epoch [990/1000] Lr: 5.0e-06, Train loss: 0.0011, Valid loss: 0.1850, Train acc: 0.96, Valid acc: 0.93
Epoch [1000/1000] Lr: 5.0e-06, Train loss: 0.0010, Valid loss: 0.1846, Train acc: 0.96, Valid acc: 0.93

Execution Time for Fold: 79.86 mins

<9> {'l1_param': 0.0001, 'learning

Epoch [710/1000] Lr: 5.0e-09, Train loss: 0.0004, Valid loss: 0.1406, Train acc: 0.99, Valid acc: 0.94
Epoch [720/1000] Lr: 5.0e-09, Train loss: 0.0004, Valid loss: 0.1403, Train acc: 0.99, Valid acc: 0.94
Epoch [730/1000] Lr: 5.0e-09, Train loss: 0.0004, Valid loss: 0.1401, Train acc: 0.99, Valid acc: 0.94
Epoch [740/1000] Lr: 5.0e-09, Train loss: 0.0004, Valid loss: 0.1402, Train acc: 0.99, Valid acc: 0.94
Epoch [750/1000] Lr: 5.0e-09, Train loss: 0.0004, Valid loss: 0.1402, Train acc: 0.99, Valid acc: 0.94
Epoch [760/1000] Lr: 5.0e-09, Train loss: 0.0004, Valid loss: 0.1403, Train acc: 0.99, Valid acc: 0.94
Epoch [770/1000] Lr: 5.0e-09, Train loss: 0.0004, Valid loss: 0.1402, Train acc: 0.99, Valid acc: 0.94
Epoch [780/1000] Lr: 5.0e-09, Train loss: 0.0004, Valid loss: 0.1405, Train acc: 0.99, Valid acc: 0.94
Epoch [790/1000] Lr: 5.0e-09, Train loss: 0.0004, Valid loss: 0.1403, Train acc: 0.99, Valid acc: 0.94
Epoch [800/1000] Lr: 5.0e-09, Train loss: 0.0004, Valid loss: 0.1402, Tra

Epoch [500/1000] Lr: 3.0e-09, Train loss: 0.0005, Valid loss: 0.1393, Train acc: 0.99, Valid acc: 0.94
Epoch [510/1000] Lr: 3.0e-09, Train loss: 0.0005, Valid loss: 0.1393, Train acc: 0.99, Valid acc: 0.94
Epoch [520/1000] Lr: 3.0e-09, Train loss: 0.0005, Valid loss: 0.1394, Train acc: 0.99, Valid acc: 0.94
Epoch [530/1000] Lr: 3.0e-09, Train loss: 0.0005, Valid loss: 0.1390, Train acc: 0.99, Valid acc: 0.94
Epoch [540/1000] Lr: 3.0e-09, Train loss: 0.0005, Valid loss: 0.1391, Train acc: 0.99, Valid acc: 0.94
Epoch [550/1000] Lr: 3.0e-09, Train loss: 0.0005, Valid loss: 0.1391, Train acc: 0.98, Valid acc: 0.94
Epoch [560/1000] Lr: 3.0e-09, Train loss: 0.0005, Valid loss: 0.1391, Train acc: 0.99, Valid acc: 0.94
Epoch [570/1000] Lr: 3.0e-09, Train loss: 0.0004, Valid loss: 0.1391, Train acc: 0.99, Valid acc: 0.94
Epoch [580/1000] Lr: 3.0e-09, Train loss: 0.0005, Valid loss: 0.1389, Train acc: 0.99, Valid acc: 0.94
Epoch [590/1000] Lr: 3.0e-09, Train loss: 0.0005, Valid loss: 0.1390, Tra

Epoch [290/1000] Lr: 1.0e-05, Train loss: 0.0017, Valid loss: 0.2127, Train acc: 0.92, Valid acc: 0.93
Epoch [300/1000] Lr: 1.0e-05, Train loss: 0.0016, Valid loss: 0.2098, Train acc: 0.92, Valid acc: 0.93
Epoch [310/1000] Lr: 1.0e-05, Train loss: 0.0015, Valid loss: 0.2079, Train acc: 0.93, Valid acc: 0.93
Epoch [320/1000] Lr: 1.0e-05, Train loss: 0.0016, Valid loss: 0.2055, Train acc: 0.93, Valid acc: 0.93
Epoch [330/1000] Lr: 1.0e-05, Train loss: 0.0015, Valid loss: 0.2032, Train acc: 0.93, Valid acc: 0.93
Epoch [340/1000] Lr: 1.0e-05, Train loss: 0.0015, Valid loss: 0.2018, Train acc: 0.93, Valid acc: 0.93
Epoch [350/1000] Lr: 1.0e-05, Train loss: 0.0015, Valid loss: 0.1993, Train acc: 0.93, Valid acc: 0.93
Epoch [360/1000] Lr: 1.0e-05, Train loss: 0.0014, Valid loss: 0.1975, Train acc: 0.93, Valid acc: 0.93
Epoch [370/1000] Lr: 1.0e-05, Train loss: 0.0014, Valid loss: 0.1960, Train acc: 0.94, Valid acc: 0.93
Epoch [380/1000] Lr: 1.0e-05, Train loss: 0.0014, Valid loss: 0.1945, Tra

Epoch [80/1000] Lr: 5.0e-06, Train loss: 0.0038, Valid loss: 0.3877, Train acc: 0.76, Valid acc: 0.82
Epoch [90/1000] Lr: 5.0e-06, Train loss: 0.0036, Valid loss: 0.3724, Train acc: 0.78, Valid acc: 0.83
Epoch [100/1000] Lr: 5.0e-06, Train loss: 0.0035, Valid loss: 0.3596, Train acc: 0.78, Valid acc: 0.83
Epoch [110/1000] Lr: 5.0e-06, Train loss: 0.0034, Valid loss: 0.3483, Train acc: 0.79, Valid acc: 0.84
Epoch [120/1000] Lr: 5.0e-06, Train loss: 0.0033, Valid loss: 0.3377, Train acc: 0.81, Valid acc: 0.85
Epoch [130/1000] Lr: 5.0e-06, Train loss: 0.0031, Valid loss: 0.3285, Train acc: 0.82, Valid acc: 0.85
Epoch [140/1000] Lr: 5.0e-06, Train loss: 0.0031, Valid loss: 0.3198, Train acc: 0.83, Valid acc: 0.85
Epoch [150/1000] Lr: 5.0e-06, Train loss: 0.0030, Valid loss: 0.3126, Train acc: 0.83, Valid acc: 0.86
Epoch [160/1000] Lr: 5.0e-06, Train loss: 0.0029, Valid loss: 0.3057, Train acc: 0.84, Valid acc: 0.86
Epoch [170/1000] Lr: 5.0e-06, Train loss: 0.0028, Valid loss: 0.2992, Train